In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense, Dropout

In [2]:
# Load IMDB Dataset
vocab_size = 10000
max_length = 200
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

In [3]:
# Padding sequences (changed padding to 'pre')
x_train = pad_sequences(x_train, maxlen=max_length, padding='pre', truncating='pre')
x_test = pad_sequences(x_test, maxlen=max_length, padding='pre', truncating='pre')

In [4]:
# Model Definition
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),
    GRU(32),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [5]:
# Compile the Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
# Train the Model (reduced epochs to 3)
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 76s 185ms/step - accuracy: 0.7234 - loss: 0.5229 - val_accuracy: 0.8699 - val_loss: 0.3072
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 76s 194ms/step - accuracy: 0.9107 - loss: 0.2369 - val_accuracy: 0.8666 - val_loss: 0.3151
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 77s 198ms/step - accuracy: 0.9441 - loss: 0.1566 - val_accuracy: 0.8443 - val_loss: 0.3816


In [7]:
# Evaluate the Model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 32ms/step - accuracy: 0.8436 - loss: 0.3887
Test Accuracy: 0.8443


In [8]:
# Sentiment Prediction
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [9]:
def encode_review(text):
    words = text.lower().split()
    encoded = [word_index.get(word, 2) for word in words]  # Use 2 for unknown words
    return pad_sequences([encoded], maxlen=max_length, padding='pre', truncating='pre')

In [10]:
# Test with a sample review (different review text)
review = "The movie was terrible, I regret watching it."
encoded_review = encode_review(review)

In [11]:
prediction = model.predict(encoded_review)[0][0]
sentiment = "Positive" if prediction > 0.5 else "Negative"

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step


In [12]:
print(f"Review: {review}")
print(f"Predicted Sentiment: {sentiment} (Confidence: {prediction:.4f})")

Review: The movie was terrible, I regret watching it.
Predicted Sentiment: Negative (Confidence: 0.4462)
